<a href="https://colab.research.google.com/github/cras-lab/Finance/blob/main/SamsungCanPredictKOSPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

필요한 모듈을 임포트 한다.

In [5]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

데이터 수집 기간을 설정한다.

In [6]:
# 데이터 수집기간 설정
start_date = '2000-01-01'
end_date = '2023-12-31'

Yahoo finance에서 삼성전자와 KOSPI 데이터를 다운로드 한다.

In [18]:
# 삼성전자와 KOSPI 데이터 다운로드
samsung = yf.download('005930.KS', start=start_date, end=end_date, progress=False)
kospi = yf.download('^KS11', start=start_date, end=end_date, progress=False)

종가만 따로 추출한다.

In [8]:
# 필요한 열(종가)만 추출
samsung_close = samsung['Close']
kospi_close = kospi['Close']

각각 일변화율을 추출해 별도로 보관한다.

In [9]:
# 상승/하락 계산을 위한 일일 변화율 계산
samsung_diff = samsung_close.pct_change().dropna()
kospi_diff = kospi_close.pct_change().dropna()

두 변화율 데이터를 병합하여, 하나의 데이터프레임으로 만든다.

In [10]:
# 데이터 병합
data = pd.DataFrame({'Samsung': samsung_diff, 'KOSPI': kospi_diff}).dropna()

연도별로 상승과 하락을 분석해서, 총 동반 상승횟수와 동반 하락 횟수를 추출한다.

In [11]:
# 연도별로 분석
years = range(2000, 2024)
results = []

for year in years:
    yearly_data = data[data.index.year == year]
    if len(yearly_data) == 0:
        continue

    samsung_up = yearly_data['Samsung'] > 0
    kospi_up = yearly_data['KOSPI'] > 0

    both_up = (samsung_up & kospi_up).sum()
    both_down = (~samsung_up & ~kospi_up).sum()
    samsung_up_kospi_down = (samsung_up & ~kospi_up).sum()
    samsung_down_kospi_up = (~samsung_up & kospi_up).sum()
    total_days = len(yearly_data)

    same_direction_ratio = (both_up + both_down) / total_days * 100
    opposite_direction_ratio = (samsung_up_kospi_down + samsung_down_kospi_up) / total_days * 100

    correlation, _ = pearsonr(yearly_data['Samsung'], yearly_data['KOSPI'])

    results.append({
        'Year': year,
        'Same Direction Ratio': same_direction_ratio,
        'Opposite Direction Ratio': opposite_direction_ratio,
        'Pearson Correlation': correlation
    })

편의상 전체 결과를 데이터프레임에 저장한다.

In [12]:
# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

전체 기간 중 동반상승, 동반하락, 반대 움직임 등을 구분하여, 각각 저장한다.

In [13]:
# 전체 기간에 대한 분석
samsung_up = data['Samsung'] > 0
kospi_up = data['KOSPI'] > 0

both_up = (samsung_up & kospi_up).sum()
both_down = (~samsung_up & ~kospi_up).sum()
samsung_up_kospi_down = (samsung_up & ~kospi_up).sum()
samsung_down_kospi_up = (~samsung_up & kospi_up).sum()
total_days = len(data)

same_direction_ratio = (both_up + both_down) / total_days * 100
opposite_direction_ratio = (samsung_up_kospi_down + samsung_down_kospi_up) / total_days * 100

correlation, _ = pearsonr(data['Samsung'], data['KOSPI'])

방향성에 대한 비율 분석을 출력한다.

In [ ]:
# 전체 결과 출력
print(f"Overall Same direction ratio: {same_direction_ratio:.2f}%")
print(f"Overall Opposite direction ratio: {opposite_direction_ratio:.2f}%")
print(f"Overall Pearson correlation: {correlation:.2f}")

각 연도별 값을 출력해 본다.

In [ ]:
# 연도별 결과 출력
print(results_df)

그래프로 시각화 해본다.

In [ ]:
# 그래프 시각화
plt.figure(figsize=(10, 10))

plt.subplot(2, 1, 1)
plt.plot(results_df['Year'], results_df['Same Direction Ratio'], label='Same Direction Ratio')
plt.plot(results_df['Year'], results_df['Opposite Direction Ratio'], label='Opposite Direction Ratio')
plt.legend()
plt.title('Yearly Movement Ratios')

plt.subplot(2, 1, 2)
plt.plot(results_df['Year'], results_df['Pearson Correlation'], label='Pearson Correlation')
plt.legend()
plt.title('Yearly Pearson Correlation')

plt.tight_layout()
plt.show()
